In [1]:
import numpy as np
import pandas as pd 
import os
from sktime.datasets import load_from_arff_to_dataframe
from pyts.utils import windowed_view
from scipy.signal import find_peaks
from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support

# Feature extraction

In [2]:
def feature_extraction(data, apply_fft=True):

    def aux_extract(data):
        features = []
        features.append(data.mean(1))
        features.append(data.std(1))
        features.append(np.abs(data - data.mean(1)[:, np.newaxis, :]).mean(1))
        features.append(data.min(1))
        features.append(data.max(1))
        features.append(data.max(1) - data.min(1))

        features.append(np.median(data, axis=1))
        features.append(np.median(np.abs(data - np.median(data,1)[:, np.newaxis, :]),1))
        features.append(np.subtract(*np.percentile(data,[75, 25],1)))
        features.append(np.count_nonzero(data < 0, 1))
        features.append(np.count_nonzero(data >= 0, 1))
        features.append(np.count_nonzero(data > data.mean(1)[:, np.newaxis, :], 1))

        features.append(np.apply_along_axis(lambda x: len(find_peaks(x)[0]), 1, data))
        features.append(np.nan_to_num(skew(data,axis=1)))
        features.append(np.nan_to_num(kurtosis(data, axis=1)))
        features.append(np.mean(data**2,axis=1))
        features.append(np.mean(np.sqrt(np.sum(data**2,axis=-1)),axis=1)[:, np.newaxis])
        features.append(np.sum(np.mean(np.abs(data),axis=1),axis=1)[:, np.newaxis])

        features = np.concatenate(features,axis=1)

        return features
    
    seq_len = data.shape[1]
    fft_data = np.abs(np.fft.fft(data,axis=1))[:, 1:(seq_len//2+1), :]

    result = aux_extract(data)
    if apply_fft:
        result = np.concatenate([result, aux_extract(fft_data)],axis=1)

    return result

In [11]:
def feature_extraction_window(data, window_size=10, window_step=3, apply_fft=True):

    data = np.stack(
        [
            windowed_view(data[:,:,idx], window_size, window_step)
            for idx in range(data.shape[-1])
        ],
        axis=-1
    )

    def aux_extract(data, prefix=""):
        n_samples, n_windows, _, dims = data.shape

        features = []
        feature_names = []
        features.append(data.mean(2).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}mean{i}_dim{j}" for i in range(n_windows) for j in range(dims)])
    
        features.append(data.std(2).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}std{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(np.abs(data - data.mean(2)[:, :, np.newaxis, :]).mean(2).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}mae{i}_dim{j}" for i in range(n_windows) for j in range(dims)])
    
        features.append(data.min(2).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}min{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(data.max(2).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}max{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(data.max(2).reshape(n_samples,-1) - data.min(2).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}extremadiff{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(np.median(data, axis=2).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}median{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(np.median(np.abs(data - np.median(data,2)[:, :, np.newaxis, :]),2).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}mad{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(np.subtract(*np.percentile(data,[75, 25],2)).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}iqr{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(np.count_nonzero(data < 0, 2).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}neg{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(np.count_nonzero(data >= 0, 2).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}pos{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(np.count_nonzero(data > data.mean(2)[:, :, np.newaxis, :], 2).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}above_mean{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(np.apply_along_axis(lambda x: len(find_peaks(x)[0]), 2, data).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}peaks{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(np.nan_to_num(skew(data,axis=2).reshape(n_samples,-1)))
        feature_names.extend([f"{prefix}skew{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(np.nan_to_num(kurtosis(data, axis=2).reshape(n_samples,-1)))
        feature_names.extend([f"{prefix}krt{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(np.mean(data**2,axis=2).reshape(n_samples,-1))
        feature_names.extend([f"{prefix}energy{i}_dim{j}" for i in range(n_windows) for j in range(dims)])

        features.append(np.mean(np.sqrt(np.sum(data**2,axis=-1)),axis=2)[:, :, np.newaxis].reshape(n_samples,-1))
        feature_names.extend([f"{prefix}acc{i}" for i in range(n_windows)])

        features.append(np.sum(np.mean(np.abs(data),axis=2),axis=2)[:, :, np.newaxis].reshape(n_samples,-1))
        feature_names.extend([f"{prefix}sma{i}" for i in range(n_windows)])

        features = np.concatenate(features,axis=1)

        return features, feature_names
    
    seq_len = data.shape[2]
    fft_data = np.abs(np.fft.fft(data, axis=2))[:, :, 1:(seq_len//2+1), :]
    result,res_names = aux_extract(data)
    if apply_fft:
        fft_res, fft_feat = aux_extract(fft_data, "fft_")
        result = np.concatenate([result, fft_res],axis=1)
        res_names = res_names + fft_feat
    return result, res_names

# Read data + convert numpy

In [12]:
DATASET = "mitbih" # ["racketsports", "mitbih", "ptbdb"]
DATA_PATH = "data"
LABEL_COL = 187

In [13]:
def dataframe2numpy(X):
    N = len(X)
    S = len(X.iloc[0][0])
    H = len(X.columns)
    return np.stack(X.values.reshape(-1)).reshape(N,S,H)

In [14]:
if DATASET == "racketsports":

    X_train, y_train = load_from_arff_to_dataframe(
        os.path.join(DATA_PATH, "RacketSports/RacketSports_TRAIN.arff")
    )

    X_test, y_test = load_from_arff_to_dataframe(
        os.path.join(DATA_PATH, "RacketSports/RacketSports_TEST.arff")
    )

    rs_train = dataframe2numpy(X_train)
    rs_test = dataframe2numpy(X_test)

    label2id = {el:i for i, el in enumerate(list(np.unique(y_train)))}

    target_train = pd.Series(y_train).apply(lambda x:label2id[x]).values
    target_test = pd.Series(y_test).apply(lambda x:label2id[x]).values

    train_features = feature_extraction_window(rs_train)
    test_features = feature_extraction_window(rs_test)

elif DATASET == "mitbih":

    mit_bih_train = pd.read_csv(os.path.join("data","ECG","mitbih_train.csv"),header=None)
    target_train = mit_bih_train[LABEL_COL].copy().values
    mit_bih_train.drop(LABEL_COL,axis=1,inplace=True)

    mit_bih_test = pd.read_csv(os.path.join("data","ECG","mitbih_test.csv"),header=None)
    target_test = mit_bih_test[LABEL_COL].copy().values
    mit_bih_test.drop(LABEL_COL,axis=1,inplace=True)

    mitbih_train = mit_bih_train.values
    mitbih_test = mit_bih_test.values

    train_features,feature_names = feature_extraction_window(mitbih_train[:,:,np.newaxis],20,8)
    test_features,_ = feature_extraction_window(mitbih_test[:,:,np.newaxis],20,8)

elif DATASET == "ptbdb":
    
    abnormal = pd.read_csv(os.path.join("data","ECG","ptbdb_abnormal.csv"),header=None)
    normal = pd.read_csv(os.path.join("data","ECG","ptbdb_normal.csv"),header=None)

    train_abn, test_abn = train_test_split(abnormal, test_size=0.2, random_state=42)
    train_nor, test_nor = train_test_split(normal, test_size=0.2, random_state=42)

    ptbdb_train = pd.concat([train_abn, train_nor]).sample(frac=1, random_state=42).reset_index(drop=True)
    ptbdb_test = pd.concat([test_abn, test_nor]).sample(frac=1, random_state=42).reset_index(drop=True)

    target_train = ptbdb_train[LABEL_COL].copy()
    ptbdb_train.drop(LABEL_COL,axis=1,inplace=True)

    target_test = ptbdb_test[LABEL_COL].copy()
    ptbdb_test.drop(LABEL_COL,axis=1,inplace=True)

    ptbdb_train = ptbdb_train.values
    ptbdb_test = ptbdb_test.values

    train_features = feature_extraction_window(ptbdb_train[:,:,np.newaxis],20,8)
    test_features = feature_extraction_window(ptbdb_test[:,:,np.newaxis],20,8)
    


/tmp/ipykernel_10712/4162814476.py:55: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(np.nan_to_num(skew(data,axis=2).reshape(n_samples,-1)))
/tmp/ipykernel_10712/4162814476.py:58: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(np.nan_to_num(kurtosis(data, axis=2).reshape(n_samples,-1)))
/tmp/ipykernel_10712/4162814476.py:55: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features.append(np.nan_to_num(skew(data,axis=2).reshape(n_samples,-1)))
/tmp/ipykernel_10712/4162814476.py:58: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occ

# Feature selection & standardization

In [17]:
selecter = VarianceThreshold(2)
scaler = StandardScaler()

In [18]:
selecter = selecter.fit(train_features)
selected_train = selecter.transform(train_features)
selected_test = selecter.transform(test_features)

In [19]:
scaler = scaler.fit(selected_train)
scaled_train = scaler.transform(selected_train)
scaled_test = scaler.transform(selected_test)

In [11]:
scaled_train.shape

(87554, 74)

In [24]:
idxs = selecter.get_support(indices=True)
selected_feats = [elem for i,elem in enumerate(feature_names) if i in idxs]

# Hyperparameter search

In [12]:
MODEL_NAME = "svm" # ["svm", "rforest", "xgb"]
MODEL = None
MODEL_PARAMS = None

In [13]:
if MODEL_NAME == "svm":
    MODEL = SVC()
    MODEL_PARAMS = {
        "kernel":["linear","poly","rbf"],
        "C":[1.0,5.0,10.0,20.0]
    }

elif MODEL_NAME == "rforest":
    MODEL = RandomForestClassifier()
    MODEL_PARAMS = {
        "n_estimators":range(40,301,20),
        "max_depth":[3, 5, 8, 12],
        "max_samples":[0.4, 0.7, 1.0]
    }

elif MODEL_NAME == "xgb":
    MODEL = XGBClassifier()
    MODEL_PARAMS = {
        "n_estimators":range(40,101,20),
        "max_depth":[3, 5, 8],
        "learning_rate":[1e-3, 1e-2, 1e-1]
    }

else:
    print("Untested model")

In [14]:
clf = GridSearchCV(MODEL, MODEL_PARAMS,n_jobs=-1)

In [15]:
clf = clf.fit(scaled_train, target_train)

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

In [109]:
clf.best_params_

{'max_depth': 12, 'max_samples': 1.0, 'n_estimators': 240}

# Evaluation

In [112]:
results_df = []

for i, params in enumerate(pd.DataFrame(clf.cv_results_)["params"]):
    if MODEL_NAME == "svm":
        curr_model = SVC(**params)
    elif MODEL_NAME == "rforest":
        curr_model = RandomForestClassifier(**params)
    elif MODEL_NAME == "xgb":
        curr_model = XGBClassifier(**params)
    
    curr_model = curr_model.fit(scaled_train, target_train)

    y_true = target_test
    y_pred = curr_model.predict(scaled_test)


    param_str = ", ".join([f"{k}={v}" for k,v in params.items()])
    row = [param_str]
    
    accuracy = accuracy_score(y_true, y_pred)
    row.append(accuracy)
    res = list(map(np.mean, precision_recall_fscore_support(y_true, y_pred, zero_division=0)))
    res2 = list(map(np.std, precision_recall_fscore_support(y_true, y_pred, zero_division=0)))
    
    mean_std = zip(res[:-1], res2[:-1])
    for mean_std_tuple in mean_std:
        row.extend(list(mean_std_tuple))

    results_df.append(row)
    print(i)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167


In [114]:
columns = [
    "Hyperparameters",
    "Accuracy",
    "Mean precision",
    "Std precision",
    "Mean recall",
    "Std recall",
    "Mean f1",
    "Std f1"
]

df = pd.DataFrame(results_df,columns=columns).round(3)
df.to_excel(f"{DATASET}_{MODEL_NAME}_results.xlsx")
# df

In [115]:
res = classification_report(target_test, clf.predict(scaled_test))
print(res)

              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97     18118
         1.0       0.99      0.49      0.65       556
         2.0       0.96      0.72      0.82      1448
         3.0       0.91      0.31      0.46       162
         4.0       1.00      0.88      0.94      1608

    accuracy                           0.95     21892
   macro avg       0.96      0.68      0.77     21892
weighted avg       0.95      0.95      0.95     21892

